In [1]:
from tensorflow import keras
import numpy as np

f_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = f_mnist.load_data()

x_train_full = np.reshape(x_train_full, (x_train_full.shape[0], x_train_full.shape[1] * x_train_full.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

print(f"Training dataset: {x_train_full.shape}, {x_train_full.dtype}")
print(f"Testing dataset: {x_test.shape}, {x_test.dtype}")

x_train = x_train_full / 255.0
y_train = y_train_full

2024-03-05 16:23:38.852524: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 16:23:38.854476: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 16:23:38.890544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 16:23:38.890573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 16:23:38.891417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Training dataset: (60000, 784), uint8
Testing dataset: (10000, 784), uint8


In [2]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [3]:
class_names = [
    'T-shirt',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

class_names[y_train[0]]

'Ankle boot'

In [4]:
from sklearn.decomposition import PCA


pca = PCA(n_components=392)
x_train_red = pca.fit_transform(x_train_scaled)
x_test_red = pca.transform(x_test_scaled)

print(f"Training dataset: {x_train_red.shape}, {x_train_red.dtype}")
print(f"Testing dataset: {x_test_red.shape}, {x_test_red.dtype}")

Training dataset: (60000, 392), float64
Testing dataset: (10000, 392), float64


In [5]:
from sklearn.random_projection import GaussianRandomProjection


rp = GaussianRandomProjection(n_components=392)
x_train_rp = rp.fit_transform(x_train_scaled)
x_test_rp = rp.transform(x_test_scaled)

print(f"Training dataset: {x_train_rp.shape}, {x_train_rp.dtype}")
print(f"Testing dataset: {x_train_rp.shape}, {x_train_rp.dtype}")

Training dataset: (60000, 392), float64
Testing dataset: (60000, 392), float64


In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=392)
x_train_lda = rp.fit_transform(x_train_scaled)
x_test_lda = rp.transform(x_test_scaled)

print(f"Training dataset: {x_train_lda.shape}, {x_train_lda.dtype}")
print(f"Testing dataset: {x_train_lda.shape}, {x_train_lda.dtype}")

Training dataset: (60000, 392), float64
Testing dataset: (60000, 392), float64


In [7]:
from sklearn.tree import DecisionTreeClassifier


classifier_tree = DecisionTreeClassifier()

In [8]:
from sklearn.neighbors import KNeighborsClassifier


classifier_kmeans = KNeighborsClassifier()

In [9]:
from sklearn.svm import LinearSVC


classifier_svc = LinearSVC(dual=True)

In [10]:
from sklearn.linear_model import LogisticRegression


classifier_regression = LogisticRegression(solver='lbfgs')

In [11]:
classifying_models = {
    'Decision Tree': classifier_tree,
    'K-Means': classifier_kmeans,
    'C-Support Vector': classifier_svc,
    'Logistic Regression (aka logit, MaxEnt)': classifier_regression
}

classifying_analyze = {
    'Original dataset': [],
    'PCA': [],
    'Random Projection': [],
    'LDA': []
}

classifying_analyze_time = {
    'Original dataset': [],
    'PCA': [],
    'Random Projection': [],
    'LDA': []
}

In [12]:
import time


for model_name, model in classifying_models.items():
    start = time.time()
    model.fit(x_train, y_train)
    end = time.time()
    
    classifying_analyze_time['Original dataset'].append(end-start)
    classifying_analyze['Original dataset'].append(model.score(x_test, y_test))

    start = time.time()
    y_pred = model.fit(x_train_red, y_train)
    end = time.time()
    
    classifying_analyze_time['PCA'].append(end-start)
    classifying_analyze['PCA'].append(model.score(x_test_red, y_test))

    start = time.time()
    model.fit(x_train_rp, y_train)
    end = time.time()
    
    classifying_analyze_time['Random Projection'].append(end-start)
    classifying_analyze['Random Projection'].append(model.score(x_test_rp, y_test))

    start = time.time()
    model.fit(x_train_lda, y_train)
    end = time.time()
    
    classifying_analyze_time['LDA'].append(end-start)
    classifying_analyze['LDA'].append(model.score(x_test_lda, y_test))

/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/linear_model/_logisti

In [13]:
classifying_analyze, classifying_analyze_time

({'Original dataset': [0.3864, 0.3211, 0.7642, 0.7505],
  'PCA': [0.2185, 0.3318, 0.7441, 0.7414],
  'Random Projection': [0.2248, 0.3291, 0.7545, 0.7355],
  'LDA': [0.201, 0.2066, 0.7346, 0.7357]},
 {'Original dataset': [65.54739665985107,
   0.04028034210205078,
   360.8344283103943,
   19.75650382041931],
  'PCA': [171.7483582496643,
   0.12505626678466797,
   465.5819401741028,
   12.332250356674194],
  'Random Projection': [155.68603801727295,
   0.0361180305480957,
   477.61306262016296,
   12.360661268234253],
  'LDA': [138.44100522994995,
   0.0406498908996582,
   494.57153367996216,
   11.708004474639893]})

In [14]:
classifying_analyze['Model'] = ['Decision Tree', 'K-Means', 'C-Support Vector', 'Logistic Regression (aka logit, MaxEnt)']
classifying_analyze_time['Model'] = ['Decision Tree', 'K-Means', 'C-Support Vector', 'Logistic Regression (aka logit, MaxEnt)']

In [15]:
import pandas as pd


data = classifying_analyze
df = pd.DataFrame.from_dict(data)

df.set_index('Model')

,Original dataset,PCA,Random Projection,LDA
Model,,,,
Decision Tree,0.3864,0.2185,0.2248,0.2010
K-Means,0.3211,0.3318,0.3291,0.2066
C-Support Vector,0.7642,0.7441,0.7545,0.7346
"Logistic Regression (aka logit, MaxEnt)",0.7505,0.7414,0.7355,0.7357


In [16]:
data = classifying_analyze_time
df = pd.DataFrame.from_dict(data)

df.set_index('Model')

,Original dataset,PCA,Random Projection,LDA
Model,,,,
Decision Tree,65.547397,171.748358,155.686038,138.441005
K-Means,0.040280,0.125056,0.036118,0.040650
C-Support Vector,360.834428,465.581940,477.613063,494.571534
"Logistic Regression (aka logit, MaxEnt)",19.756504,12.332250,12.360661,11.708004
